# Brainstorm



Constitution Ideas:
* target-only (has mention; control)
* hateful keyword
*  -->

google trends: "black lives matter":
* May 3 - Dec 26, 2020 (1% to 1%)
* May 17 - Aug 22 (spike onset to decay; has longer tail)

google trends: "looting":
* May 17 - June 20 (spike onset to decay; no tail)

google trends: "blue lives matter":
* May 17 - July 11 (spike onset to decay; long tail)

Annotation Strategy:
1. Select tweets by oversampling tweets with BLM or "hate" keywords
2. 2 annnotators read texts, sample 50 tweets, go over and discuss with me, revisit 50 tweets for changes, go over, done
3. annotators annotate 3k tweets
4. Note inter-rater score and Cohen's Kappa score
5. Select data classified the same by BOTH annotators


# Add and clean dataset

**Dataset info:**
* 9165 tweets
* data collection period: 05/27 to 7/26 (2 months)
* excluded users <150 followers (reduce bot spam tweets)
* hate-speech specific tweets targeted via hashtags: BLM, ChangeTheSystem, AtlantaProtest, JusticeForGeorgeFloyd, BlueLivesMatter

**Annotation criteria**
* mention of hateful text
* mention of non-hateful text
* 5 undergrad/grad annotators, 100% agreement of at least 2/3 annotators selects tweet for final annotation, if agreement of last 2 annotators then final labeled, otherwise remove tweet from consideration



In [20]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import nltk
import pandas as pd
import ast
from IPython.display import display
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
url = 'https://raw.githubusercontent.com/saadiahmadx/IW23/main/TweetBLM.csv'
df = pd.read_csv(url)
updated_df = df

info. gathering + basic dataset cleaning 

In [24]:
print ('n =', df['hate label'].count(), '\n')
print(df['hate label'].value_counts(normalize=True), '(normalized) \n')
print(df['source'].value_counts(normalize=True), '(normalized) \n')

#convert user_created dates to numeric
updated_df['user_created'] = pd.to_datetime(df['user_created'], errors='coerce', dayfirst=True)

#collection period, 1mo before, 6mo, 1yr, 5yr
custom_bins = ['2020-08-10 11:00:00', '2020-05-26 00:00:00', '2020-04-26 00:00:00', '2019-11-26 00:00:00', '2019-05-26 00:00:00', '2015-05-26 00:00:00']
custom_bins = pd.to_datetime(custom_bins.reverse())
print(updated_df['user_created'].value_counts(normalize=True, bins=12), '(normalized) \n')

print(updated_df['hashtags'].value_counts(normalize=True), '(normalized) \n')
# def convert_hashtag_string(s):
#   t = ast.literal_eval(s)
#   t = [n.strip() for n in t]
#   return t
# all_hashtags = []
# for index, row in df['hashtags'].iteritems():
#   print(row)
#   if (row):
#     h = convert_hashtag_string(str(row).replace('\'', '"'))
#     all_hashtags = all_hashtags + h
# df_hashtag = pd.Series(all_hashtags)
# print(df_hashtag.value_counts(normalize=True))

n = 9165 

0    0.663502
1    0.336498
Name: hate label, dtype: float64 (normalized) 

Twitter for iPhone     0.338791
Twitter for Android    0.311722
Twitter Web App        0.274722
Twitter for iPad       0.046169
TweetDeck              0.004802
                         ...   
SYSTHUB                0.000109
WordPress.com          0.000109
Biden Cabinet App      0.000109
Echofon                0.000109
dlvr.it                0.000109
Name: source, Length: 79, dtype: float64 (normalized) 

(2019-06-17 15:05:00, 2020-08-10 11:00:00]              0.231424
(2009-02-11 03:50:00, 2010-04-06 23:45:00]              0.115330
(2018-04-23 19:10:00, 2019-06-17 15:05:00]              0.092526
(2011-05-31 19:40:00, 2012-07-24 15:35:00]              0.089689
(2016-01-05 03:20:00, 2017-02-27 23:15:00]              0.083797
(2017-02-27 23:15:00, 2018-04-23 19:10:00]              0.081397
(2013-09-17 11:30:00, 2014-11-11 07:25:00]              0.075505
(2012-07-24 15:35:00, 2013-09-17 11:30:00]        

tweet preprocessing

In [25]:
import re, string, unicodedata
import contractions
import nltk
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.util import ngrams

text='RT @AgainWoodsy: 🍏🍏 Don\'t like mentally retarded & weird @JoeBiden #BlackLivesMatter activists screaming like big girls in a shopping store. https://t.co/KlhxJoSoLu'

def get_media_url(tweet):
  media_link = 'https://t.co/' + re.findall("https://t.co/([a-zA-Z0-9_]{1,50})", tweet)[0]
  return media_link

def get_retweet(tweet):
  return re.findall("RT @([a-zA-Z0-9_]{1,50})", tweet)

def get_mentions(tweet):
  return re.findall("@([a-zA-Z0-9_]{1,50})", tweet)

def get_hashtags(tweet):
  return re.findall("#([a-zA-Z0-9_]{1,50})", tweet)

def remove_url(text):
  return re.sub(r"https?:[^\s]+", "", text)

def remove_contractions(text):
  return contractions.fix(text)

def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

def process_tweet(tweet):
  p_tweet = tweet
  p_tweet = remove_url(p_tweet)
  p_tweet = remove_contractions(p_tweet)
  retweet = get_retweet(p_tweet)
  if (retweet):
    p_tweet = p_tweet.replace('RT @' + retweet[0] + ': ', '')
  p_tweet = p_tweet.lower()
  mentions = get_mentions(p_tweet)
  if (mentions):
    for i in range(len(mentions)):
      p_tweet = p_tweet.replace(' @' + mentions[i], '')
  hashtags = get_hashtags(p_tweet)
  if (hashtags):
    for i in range(len(hashtags)):
      p_tweet = p_tweet.replace(' #' + hashtags[i], '')
  
  words = nltk.word_tokenize(p_tweet)
  #print('\nwords:', words)

  words = normalize(words)
  return words

print(' Original Tweet:\n', text, '\n\n', 'Processed Tweet:\n', process_tweet(text))
print('\nMedia Link:', get_media_url(text), '\nRetweet:', get_retweet(text), '\nMentions:', get_mentions(text), '\nHashtags:', get_hashtags(text))

 Original Tweet:
 RT @AgainWoodsy: 🍏🍏 Don't like mentally retarded & weird @JoeBiden #BlackLivesMatter activists screaming like big girls in a shopping store. https://t.co/KlhxJoSoLu 

 Processed Tweet:
 ['like', 'mentally', 'retarded', 'weird', 'activists', 'screaming', 'like', 'big', 'girls', 'shopping', 'store']

Media Link: https://t.co/KlhxJoSoLu 
Retweet: ['AgainWoodsy'] 
Mentions: ['AgainWoodsy', 'JoeBiden'] 
Hashtags: ['BlackLivesMatter']


In [26]:
all_mentions = []
all_hashtags = []
all_unigrams = []
all_bigrams = []
for index, row in df['text'].iteritems():
  all_mentions += get_mentions(row.lower())
  all_hashtags += get_hashtags(row.lower())
  processed = process_tweet(row)
  all_unigrams += processed
  all_bigrams += list(ngrams(processed, 2))
  updated_df.at[index, 'text'] = processed
updated_df.drop(['hashtags', 'source', 'user_created', 'user_friends', 'user_favourites', 'is_retweet'], axis=1, inplace=True)

In [27]:
display(updated_df)

,text,hate label
0,"[know, take, others]",0
1,"[tonight, interview, tune, 8pm, preorder]",0
2,"[officers, advance, close, crowd, attempting, ...",0
3,"[wwe, hof, skin, color, former, wwe, wrestler,...",0
4,"[shaunking, going, fighting, life, rights, rea...",0
...,...,...
9160,"[watch, segregated, design, watch, video]",1
9161,"[dawnbutler, fair, always, known, men, uniform]",1
9162,"[hislifemattered, heard, offer, condolences, a...",1
9163,"[like, whole, niranga, situation, makes, retra...",1


In [28]:
df_hashtag = pd.Series(all_hashtags)
df_mentions = pd.Series(all_mentions)
df_unigrams = pd.Series(all_unigrams)
df_bigrams = pd.Series(all_bigrams)
print('Hashtags:\n\n', df_hashtag.value_counts(normalize=True), '(normalized)\n')
print('Mentions:\n\n', df_mentions.value_counts(normalize=True), '(normalized)\n')
print('Unigrams:\n\n', df_unigrams.value_counts(normalize=True), '(normalized)\n')
print('Bigrams:\n\n', df_bigrams.value_counts(normalize=True), '(normalized)\n')

Hashtags:

 blacklivesmatter    0.228274
blm                 0.141146
antifa              0.014360
portland            0.009970
portlandprotests    0.009301
                      ...   
heaux               0.000074
stra                0.000074
anklagejetzt        0.000074
justiceformbobda    0.000074
lord                0.000074
Length: 3274, dtype: float64 (normalized)

Mentions:

 realdonaldtrump    0.016663
dawnbutlerbrent    0.012922
mrandyngo          0.006574
joebiden           0.006348
youtube            0.005214
                     ...   
heartafire777      0.000113
ayecesar_          0.000113
epochtimes         0.000113
news_ntd           0.000113
kepler186f_in      0.000113
Length: 5693, dtype: float64 (normalized)

Unigrams:

 black               0.013832
blacklivesmatter    0.010665
people              0.009582
amp                 0.007669
police              0.007220
                      ...   
maryland            0.000013
renta               0.000013
westminster        

# Model: Convert to BERT embeddings